# Give Directly Take Home Assessment

## Question 1


    1. Please evaluate the data in recipients.csv and survey_attempts.csv to answer the following questions:
        1. How many recipients are in each of the four stages? Please provide the calculation(s) in the spreadsheet or code that you submit.
        2. How many surveys were successfully completed in December, 2020? Please provide the calculation(s) in the spreadsheet or code that you submit.
        3. Did you find any abnormalities in the source data? If so, how did you account for them in your analysis?

First install the required packages.

In [ ]:
!pip install -r requirements.txt

Import the required libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('QtAgg')

import statsmodels.formula.api as smf

Let's now import the two datasets and merge them

In [ ]:
recipients = pd.read_csv('recipients.csv')
attempts = pd.read_csv('survey_attempts.csv')
merged = pd.merge(recipients, attempts, on='recipient_id', how='left', indicator=True)
merged._merge.value_counts(dropna=False)


It looks like all the recipients are matched with survey attempts. We can drop the `_merge` variable

In [ ]:
merged.drop(columns='_merge', inplace=True)

A quick glance at our data

In [ ]:
merged.head(10)

Check for duplicates. We expect that `recipient_id` and `survey_id` together form a unique id.


In [ ]:
merged[merged.duplicated(subset=['recipient_id', 'survey_id'], keep=False)]

In [ ]:
There are 50 pairs of duplicates that need to be dropped.  

In [ ]:
merged = merged.drop_duplicates(subset=['recipient_id', 'survey_id'], keep='first').reset_index(drop=True)


Let's now calculate the stage variable, starting with 'Start'.

In [ ]:
mask = merged.groupby('recipient_id')['success'].any()
one_success = [mask.index[i] for i, m in enumerate(mask) if m]
# Set the value to start for those with no successful surveys. '~' negates the value 
# of the mask. In this case, ~mask means find those without any successful survey
merged.loc[~merged.recipient_id.isin(one_success), 'stage'] = 'Start'

Next, ineligible respondents.

In [ ]:
# Remove the text 'County' from the column
merged.county = merged.county.str.replace('County ', '', regex=False)
    
inABC = merged.county.isin(['A', 'B', 'C'])
recipient_noABC = merged.recipient_id[~inABC]

merged.loc[merged.recipient_id.isin(one_success) & merged.recipient_id.isin(recipient_noABC), 'stage'] = 'Ineligible'
